In [29]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
import os
import time
import torch.nn.functional as F
from torch.optim import lr_scheduler

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 12, 3, 1)
        self.conv2 = nn.Conv2d(12, 20, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(3920, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        x = F.relu(x)

        output = F.log_softmax(x, dim=1)
        return output
    
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((32,32)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((32,32)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

data_dir = 'data/' # path of the data folder, can be edited

trainset = datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train'])

testset = datasets.ImageFolder(os.path.join(data_dir, 'test'), data_transforms['test'])

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

model_ft = Net() 

model_ft = model_ft.to(device)

dataiter = iter(trainloader)

images, labels = dataiter.next()

optimizer_ft = optim.Adam(model_ft.parameters(), lr=1e-4)

criterion = nn.CrossEntropyLoss()

def train_test(model, criterion, optimizer, scheduler, num_epochs):
    time0 = time.time()
    for e in range(num_epochs):
        running_loss = 0
        for images, labels in trainloader:

            images = images.view(images.size(0), 3, 32, 32)

            optimizer.zero_grad()

            output = model(images)
            
            loss = criterion(output, labels)

            loss.backward()

            optimizer.step()

            running_loss += loss.item()
        else:
            print("Epoch " + str(e) + " training loss: " + str(running_loss/len(trainloader)))
            
    print("\nTraining Time =" ,(time.time()-time0)/60, " minutes")

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=20, gamma=0.1)

train_test(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=30)



Epoch 0 training loss: 2.2925350300030414
Epoch 1 training loss: 1.9770161892051128
Epoch 2 training loss: 1.5350338945002444
Epoch 3 training loss: 1.3177052794743194
Epoch 4 training loss: 1.2140872888981915
Epoch 5 training loss: 1.1422389785109808
Epoch 6 training loss: 1.094171814827014
Epoch 7 training loss: 1.0550559014399676
Epoch 8 training loss: 1.0246667034590422
Epoch 9 training loss: 1.0029496096853
Epoch 10 training loss: 0.9839831478814326
Epoch 11 training loss: 0.9534223621079663
Epoch 12 training loss: 0.9402108055823393
Epoch 13 training loss: 0.9213683487001513
Epoch 14 training loss: 0.9130349113488756
Epoch 15 training loss: 0.8976701567930453
Epoch 16 training loss: 0.8950115882003231
Epoch 17 training loss: 0.8777693810620542
Epoch 18 training loss: 0.8697302394838475
Epoch 19 training loss: 0.8593691056852402
Epoch 20 training loss: 0.8506905125148261
Epoch 21 training loss: 0.8401846659463098
Epoch 22 training loss: 0.8326722415906789
Epoch 23 training loss: 0

In [30]:
torch.save(model_ft, 'data/model')

In [31]:
correct, total = 0, 0
for l in range(0, 10):
    Ccount, Acount = 0, 0
    for images,labels in testloader:
        for i in range(len(labels)):
            img = images[i].view(1, 3, 32, 32)
            with torch.no_grad():
                logps = model_ft(img)


            ps = torch.exp(logps)
            probab = list(ps.numpy()[0])
            pred_label = probab.index(max(probab))
            true_label = labels.numpy()[i]
            if(true_label == l):
                Acount += 1
                total += 1
                if(true_label == pred_label):
                    Ccount += 1
                    correct += 1
    print("Class " + str(l) + "    Number Of Images Tested = " + str(Acount))
    print("           Correct count (" + str(l) + ") = " + str(Ccount))
    print("           Model Accuracy =", (Ccount/Acount))

Class 0    Number Of Images Tested = 500
           Correct count (0) = 407
           Model Accuracy = 0.814
Class 1    Number Of Images Tested = 500
           Correct count (1) = 391
           Model Accuracy = 0.782
Class 2    Number Of Images Tested = 500
           Correct count (2) = 371
           Model Accuracy = 0.742
Class 3    Number Of Images Tested = 500
           Correct count (3) = 261
           Model Accuracy = 0.522
Class 4    Number Of Images Tested = 500
           Correct count (4) = 414
           Model Accuracy = 0.828
Class 5    Number Of Images Tested = 500
           Correct count (5) = 359
           Model Accuracy = 0.718
Class 6    Number Of Images Tested = 500
           Correct count (6) = 331
           Model Accuracy = 0.662
Class 7    Number Of Images Tested = 500
           Correct count (7) = 429
           Model Accuracy = 0.858
Class 8    Number Of Images Tested = 500
           Correct count (8) = 326
           Model Accuracy = 0.652
Class 9   

In [32]:
correct/5000

0.7294

In [37]:
torch.save(model_ft.state_dict(), data_dir + 'model')

In [38]:
Model = Net()
Model.load_state_dict(torch.load('data/model'))
Model.eval()

Net(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (dropout2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=3920, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [39]:
torch.save(model_ft, data_dir + 'Model')

In [40]:
Loadmodel = torch.load('data/Model')
Loadmodel.eval()

Net(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (dropout2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=3920, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)